# Modelling neutral beam injection with BBNBI

<img src='../../figures/template.png' alt='thumbnail' width='200'/>

This example shows how to generate beam ions with BBNBI5.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from a5py import Ascot

a5 = Ascot("ascot.h5", create=True)

# Use analytical field since it is fast to interpolate
a5.data.create_input("bfield analytical iter circular", desc="ANALYTICAL")

nrho  = 101
rho   = np.linspace(0, 2, nrho).T
prof  = (1.0 - rho**(3.0/2))**3
edens = 2e21 * prof
etemp = 1e4  * np.ones((nrho, 1))
idens = 1e21 * np.reshape(prof,(nrho,1))
itemp = 1e4  * np.ones((nrho, 1))

edens[rho>=1] = 1
idens[rho>=1] = 1

plt.plot(rho,edens)

pls = {
    "nrho" : nrho, "nion" : 1, "rho" : rho,
    "anum" : np.array([2]), "znum" : np.array([1]),
    "mass" : np.array([2.014]), "charge" : np.array([1]),
    "edensity" : edens, "etemperature" : etemp,
    "idensity" : idens, "itemperature" : itemp}
a5.data.create_input("plasma_1D", **pls, desc="FLATDT")

from a5py.ascot5io.wall import wall_3D
rad  = 2.0
pol  = np.linspace(0, 2*np.pi, 181)[:-1]
wall = {"nelements":180,
        "r":6.2 + rad*np.cos(pol), "z":0.0 + rad*np.sin(pol)}
wall = wall_3D.convert_wall_2D(180, **wall)
a5.data.create_input("wall_3D", **wall, desc="ROTATED")

from a5py.ascot5io.options import Opt

opt = Opt.get_default()
opt.update({
    # Distribution output
    "ENABLE_DIST_5D":1,
    "DIST_MIN_R":4.3,        "DIST_MAX_R":8.3,       "DIST_NBIN_R":50,
    "DIST_MIN_PHI":0,        "DIST_MAX_PHI":360,     "DIST_NBIN_PHI":1,
    "DIST_MIN_Z":-2.0,       "DIST_MAX_Z":2.0,       "DIST_NBIN_Z":50,
    "DIST_MIN_PPA":-1.3e-19, "DIST_MAX_PPA":1.3e-19, "DIST_NBIN_PPA":100,
    "DIST_MIN_PPE":0,        "DIST_MAX_PPE":1.3e-19, "DIST_NBIN_PPE":50,
    "DIST_MIN_TIME":0,       "DIST_MAX_TIME":1.0,    "DIST_NBIN_TIME":1,
    "DIST_MIN_CHARGE":1,     "DIST_MAX_CHARGE":3,    "DIST_NBIN_CHARGE":1,
})
a5.data.create_input("opt", **opt, desc="SLOWINGDOWN")

from a5py.ascot5io.nbi  import NBI
inj = NBI.generate(r=8.0, phi=0.0, z=0.0, tanrad=-6.0, focallen=1.0,
                   dgrid=[0.23, 0.52], nbeamlet=1, anum=2, znum=1,
                   mass=2.0141, energy=1e6, efrac=[1.0, 0.0, 0.0],
                   power=1e6, tilt=0.1,
                   div=[np.deg2rad(0.95)/2, np.deg2rad(0.95)/2])
a5.data.create_input("nbi", **{"ninj":1, "injectors":[inj]},
                     desc="INJECTORS")

print("Inputs created")

jyjy

In [ ]:
import subprocess
subprocess.run(["./../../build/bbnbi5", "--d=\"BEAMS\""])
print("Simulation completed")